# ConversationBufferMemory

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [2]:
from langchain.memory.buffer import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.clear()

memory.save_context(
    {"input": "Hello chatbot!"}, {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Terry"}, {"output": "Nice to meet you Terry"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})

/var/folders/p8/_23vf8_97jn_kt805y1dkmmr0000gn/T/ipykernel_48631/1130096921.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: Hello chatbot!\nAI: Hello. How can I help you?\nHuman: My name is Terry\nAI: Nice to meet you Terry\nHuman: Where is Seoul?\nAI: Seoul is in Korea'}

In [3]:
from langchain.memory.buffer import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")
memory.clear()
memory.chat_memory.add_user_message("Hello chatbot!")
memory.chat_memory.add_ai_message("Hello. How can I help you?")
memory.load_memory_variables({})

{'chat_history': 'Human: Hello chatbot!\nAI: Hello. How can I help you?'}

In [4]:
from langchain.memory.buffer import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.clear()
memory.save_context(
    {"input": "Hello chatbot!"}, {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Terry"}, {"output": "Nice to meet you Terry"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})
# returning list of chat message

{'chat_history': [HumanMessage(content='Hello chatbot!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello. How can I help you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Terry', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you Terry', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Where is Seoul?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Seoul is in Korea', additional_kwargs={}, response_metadata={})]}

In [5]:
from langchain.memory.buffer import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=False)
memory.clear()
memory.save_context(
    {"input": "Hello chatbot!"}, {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Terry"}, {"output": "Nice to meet you Terry"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})
# returning chat history with single string

{'chat_history': 'Human: Hello chatbot!\nAI: Hello. How can I help you?\nHuman: My name is Terry\nAI: Nice to meet you Terry\nHuman: Where is Seoul?\nAI: Seoul is in Korea'}

# Conversational Buffer Memory for ChatLLM

In [10]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

memory = {}  

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in memory:
        memory[session_id] = InMemoryChatMessageHistory()
    return memory[session_id]

template = """You are a chatbot having a conversation with a human.

Previous conversation history:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)

model = ChatOpenAI(api_key="{YOUR_OPENAI_KEY}", model="gpt-4o-mini", temperature=0)

chain = prompt | model

conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

response = conversation.invoke(
    {"question": "hi my name is Terry"},
    config={"configurable": {"session_id": "1"}}
)
response

AIMessage(content="Hi Terry! It's nice to meet you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 35, 'total_tokens': 51, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a34aaa2-bb42-45df-9f23-fce52c921eec-0', usage_metadata={'input_tokens': 35, 'output_tokens': 16, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [12]:
conversation.invoke({"question": "hi my name is Terry"},config={"configurable": {"session_id": "1"}})
conversation.invoke({"question": "Can you recommend fun activities for me?"},config={"configurable": {"session_id": "1"}})
conversation.invoke({"question": "What is my name?"},config={"configurable": {"session_id": "1"}})

memory["1"].messages

[HumanMessage(content='hi my name is Terry', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi Terry! It's nice to meet you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 35, 'total_tokens': 51, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a34aaa2-bb42-45df-9f23-fce52c921eec-0', usage_metadata={'input_tokens': 35, 'output_tokens': 16, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
 HumanMessage(content='hi my name is Terry', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi Terry! It seems like you already introduced yourself. How can I assist you today?', additional_